In [1]:
from tensorflow import keras
import tensorflow as tf
from keras import models
import PIL
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
!pip install Pillow

## Download and explore datasets

In [2]:
!mkdir ~/.kaggle


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
!mv kaggle.json ~/.kaggle


mv: cannot stat 'kaggle.json': No such file or directory


In [4]:
!ls -a ~/.kaggle


.  ..


In [5]:
!kaggle datasets download -d aima138/17flowerclasses


Dataset URL: https://www.kaggle.com/datasets/aima138/17flowerclasses
License(s): unknown
17flowerclasses.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
! unzip 17flowerclasses.zip

Archive:  17flowerclasses.zip
replace 17flowerclasses/test/Bluebell/image_0246.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Load data with keras

In [7]:
pip install tqdm

In [8]:
pip install timm

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import timm
import gc
from torch.cuda.amp import GradScaler, autocast

# Define the device to use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define the transformations for the train and test sets
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Resize images to 128x128 pixels to save memory
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the dataset directories
train_dir = '/content/17flowerclasses/train'
test_dir = '/content/17flowerclasses/test'

# Create datasets
full_train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Split the training dataset into training and validation sets
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Reduce batch size to save memory
batch_size = 16
num_workers = 2
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

# Print some information about the dataset
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")
print(f"Number of classes: {len(full_train_dataset.classes)}")
print(f"Class names: {full_train_dataset.classes}")

# Define the neural network (using a pre-trained EfficientNet-B0 model)
model = timm.create_model('tf_efficientnet_b0_ns', pretrained=True)
model.classifier = nn.Linear(model.classifier.in_features, len(full_train_dataset.classes))
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Mixed Precision Training
scaler = GradScaler()

# Training function with gradient accumulation and accuracy calculation
def train_single_model(model, train_loader, validation_loader, criterion, optimizer, scaler, num_epochs=8, accumulation_steps=4):
    best_val_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        optimizer.zero_grad()

        for i, (inputs, labels) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
            inputs, labels = inputs.to(device), labels.to(device)

            with autocast():  # Mixed precision training
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss = loss / accumulation_steps  # Normalize the loss to account for gradient accumulation
            scaler.scale(loss).backward()

            if (i + 1) % accumulation_steps == 0:  # Perform optimizer step after accumulating gradients
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            running_loss += loss.item()

            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_acc_train = correct_train / total_train * 100.0
        print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc_train:.2f}%')

        # Validation phase
        model.eval()
        validation_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                with autocast():  # Mixed precision training
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                validation_loss += loss.item()

                # Calculate validation accuracy
                _, predicted = torch.max(outputs, 1)
                correct_val += (predicted == labels).sum().item()
                total_val += labels.size(0)

        epoch_val_loss = validation_loss / len(val_loader)
        epoch_acc_val = correct_val / total_val * 100.0
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {epoch_val_loss:.4f}, Validation Accuracy: {epoch_acc_val:.2f}%')

        # Save the best model based on validation accuracy
        if epoch_acc_val > best_val_accuracy:
            best_val_accuracy = epoch_acc_val
            best_model_wts = model.state_dict()

        # Clear cache
        torch.cuda.empty_cache()
        gc.collect()

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    return model

# Train the model
model = train_single_model(model, train_loader, val_loader, criterion, optimizer, scaler, num_epochs=8, accumulation_steps=4)

# Evaluating the model on the test set
model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

print(f"Accuracy of the model on the test images: {100 * correct_test / total_test:.2f}%")


Number of training samples: 952
Number of validation samples: 238
Number of testing samples: 170
Number of classes: 17
Class names: ['Bluebell', 'ButterCup', 'ColtsFoot', 'Cowslip', 'Crocus', 'Daffodil', 'Daisy', 'Dandelion', 'Fritillary', 'Iris', 'LilyValley', 'Pansy', 'Snowdrop', 'Sunflower', 'Tigerlily', 'WindFlower', 'tulip']


Epoch 1/8: 100%|██████████| 60/60 [09:29<00:00,  9.50s/it]

Epoch 1/8, Training Loss: 0.3994, Training Accuracy: 71.11%


Epoch 1/8, Validation Loss: 0.2103, Validation Accuracy: 97.06%


Epoch 2/8: 100%|██████████| 60/60 [09:29<00:00,  9.49s/it]

...

Epoch 7/8, Training Loss: 0.0061, Training Accuracy: 99.37%


Epoch 7/8, Validation Loss: 0.0610, Validation Accuracy: 98.74%


Epoch 8/8: 100%|██████████| 60/60 [09:35<00:00,  9.59s/it]

Epoch 8/8, Training Loss: 0.0071, Training Accuracy: 99.37%


Epoch 8/8, Validation Loss: 0.0643, Validation Accuracy: 98.74%
Accuracy of the model on the test images: 96.47%


# Keras Model

# Train the model